In [39]:
import torch
import numpy as np
import json

class PointSequenceDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        with open(data) as f:
            self.data = json.load(f)
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        seq = torch.tensor(self.data[idx]["sequence"])
        cls = torch.tensor(self.data[idx]["intersect"]).long()
        return seq, cls

dataset = PointSequenceDataset("./lecture 4/train_dataset.json")
print(dataset[0])

(tensor([[0.9160, 0.2257],
        [1.2735, 0.2941],
        [1.1046, 0.5130],
        [1.3538, 0.2501],
        [1.4257, 0.2829],
        [1.3582, 0.7018],
        [1.5891, 0.5459],
        [1.6116, 0.6819],
        [1.8069, 0.2731],
        [2.1430, 0.1446],
        [2.3039, 0.3248]]), tensor(0))


In [44]:
class RNNModel(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        self.rnn = torch.nn.Sequential(
            torch.nn.Linear(input_size + hidden_size, hidden_size),
            torch.nn.LeakyReLU(),
            torch.nn.Linear(hidden_size, hidden_size),
            torch.nn.LeakyReLU(),
            torch.nn.Linear(hidden_size, hidden_size),
        )
        self.ff = torch.nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        hidden_state = torch.zeros(x.shape[0], self.hidden_size, device=x.device)
        for t in range(x.shape[1]):
            input = torch.cat([x[:, t, :], hidden_state], dim=1)
            hidden_state = self.rnn(input)
        return self.ff(hidden_state)
    
dl = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)
model = RNNModel(2, 10, 2)

serie, cls = next(iter(dl))
print(serie.shape, cls.shape)
out = model(serie)
print(out.shape)

torch.Size([32, 11, 2]) torch.Size([32])
torch.Size([32, 2])


In [45]:
train_ds = PointSequenceDataset("./lecture 4/train_dataset.json")
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=32, shuffle=True)
model = RNNModel(2, 300, 2)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

model = model.to("cuda")

for epoch in range(20):
    for serie, cls in train_dl:
        serie = serie.to("cuda")
        cls = cls.to("cuda")
        
        out = model(serie)
        loss = loss_fn(out, cls)

        acc = (out.argmax(dim=1) == cls).float().mean()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Epoch {epoch} Loss: {loss.item()} Acc: {acc.item()}")

Epoch 0 Loss: 0.6910014748573303 Acc: 0.53125
Epoch 0 Loss: 0.6904359459877014 Acc: 0.53125
Epoch 0 Loss: 0.7011449933052063 Acc: 0.40625
Epoch 0 Loss: 0.6961795687675476 Acc: 0.46875
Epoch 0 Loss: 0.6878122091293335 Acc: 0.5625
Epoch 0 Loss: 0.6960008144378662 Acc: 0.46875
Epoch 0 Loss: 0.7045201063156128 Acc: 0.375
Epoch 0 Loss: 0.688259482383728 Acc: 0.5625
Epoch 0 Loss: 0.6857319474220276 Acc: 0.59375
Epoch 0 Loss: 0.701404869556427 Acc: 0.40625
Epoch 0 Loss: 0.6840513348579407 Acc: 0.625
Epoch 0 Loss: 0.6840056777000427 Acc: 0.625
Epoch 0 Loss: 0.7031907439231873 Acc: 0.375
Epoch 0 Loss: 0.6934614777565002 Acc: 0.5
Epoch 0 Loss: 0.6915955543518066 Acc: 0.53125
Epoch 0 Loss: 0.7053326368331909 Acc: 0.34375
Epoch 0 Loss: 0.6878023147583008 Acc: 0.5625
Epoch 0 Loss: 0.6914972066879272 Acc: 0.53125
Epoch 0 Loss: 0.6860560178756714 Acc: 0.59375
Epoch 0 Loss: 0.6955908536911011 Acc: 0.46875
Epoch 0 Loss: 0.6909419894218445 Acc: 0.53125
Epoch 0 Loss: 0.6829795837402344 Acc: 0.625
Epoch 0

In [43]:
test_ds = PointSequenceDataset("./lecture 4/test_dataset.json")
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=32, shuffle=True)

preds = []
actuals = []
for serie, cls in test_dl:
    serie = serie.to("cuda")
    cls = cls.to("cuda")
    
    out = model(serie)

    preds.append(out.argmax(dim=1).cpu().numpy())
    actuals.append(cls.cpu().numpy())

preds = np.concatenate(preds)
actuals = np.concatenate(actuals)

print((preds == actuals).mean())

0.6015
